In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.linalg import svd

from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.covariance import ShrunkCovariance, LedoitWolf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from nilearn.masking import apply_mask
from sklearn.preprocessing import StandardScaler
%matplotlib inline
import os.path as osp
from scipy.signal import detrend
import nibabel as nib
from scipy.io import loadmat
from scipy.fftpack import fft, fftshift, fftn
from scipy.signal import correlate2d

/usr/local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# pdb

In [15]:
def autocorr(x):
    result = numpy.correlate(x, x, mode='full')
    return result[result.size / 2:]


def sumN(dat):
    # sum of the all elements of the dat matrix
    return np.sum(dat[:])


def checkOrder(n_in):
    # CHECKORDER Checks the order passed to the window functions.

    w = []
    trivalwin = 0

    # Special case of negative orders:
    if n_in < 0:
        print('ERROR!!!! Order cannot be less than zero.')

    # Check if order is already an integer or empty
    # If not, round to nearest integer.
    if not n_in or n_in == np.floor(n_in):
        n_out = n_in
    else:
        n_out = np.round(n_in)
        print('WARNING: Rounding order to nearest integer.')

    # Special cases:
    if not n_out or n_out == 0:
        w = np.zeros((0, 1))  # Empty matrix: 0-by-1
        trivalwin = 1
    elif n_out == 1:
        w = 1
        trivalwin = 1

    return n_out, w, trivalwin


def parzen_win(n):
    # PARZENWIN Parzen window.
    # PARZENWIN(N) returns the N-point Parzen (de la Valle-Poussin) window in
    # a column vector.

    # Check for valid window length (i.e., n < 0)
    n, w, trivialwin = checkOrder(n)
    if trivialwin:
        return w

    # Index vectors
    k = np.arange(-(n - 1) / 2, ((n - 1) / 2) + 1)
    k1 = k[k < -(n - 1) / 4]
    k2 = k[abs(k) <= (n - 1) / 4]

    # Equation 37 of [1]: window defined in three sections
    w1 = 2 * (1 - abs(k1) / (n / 2))**3
    w2 = 1 - 6 * (abs(k2) / (n / 2))**2 + 6 * (abs(k2) / (n / 2))**3
    w = np.hstack((w1, w2, w1[::-1])).T

    return w


def entrate_sp(x, sm_window):

    # Calculate the entropy rate of a stationary Gaussian random process using
    # spectrum estimation with smoothing window

    n = x.shape

    # Normalize x_sb to be unit variance
    x_std = np.std(np.reshape(x, (np.prod(n), 1)))
    if x_std < 1e-10:
        x_std = 1e-10
    x = x / x_std

    if (sm_window == 1):

        M = [int(i) for i in np.ceil(np.array(n) / 10)]

        if (x.ndim >= 3):
            parzen_w_3 = np.zeros((2 * n[2] - 1, ))
            parzen_w_3[(n[2] - M[2] - 1):(n[2] + M[2])] = parzen_win(2 * M[2] + 1)

        if (x.ndim >= 2):
            parzen_w_2 = np.zeros((2 * n[1] - 1, ))
            parzen_w_2[(n[1] - M[1] - 1):(n[1] + M[1])] = parzen_win(2 * M[1] + 1)

        if (x.ndim >= 1):
            parzen_w_1 = np.zeros((2 * n[0] - 1, ))
            parzen_w_1[(n[0] - M[0] - 1):(n[0] + M[0])] = parzen_win(2 * M[0] + 1)

    if x.ndim == 2 and min(n) == 1:  # 1D
        xc = autocorr(x)
        xc = xc * parzen_w
        xf = fftshift(fft(xc))

    elif x.ndim == 2 and min(n) != 1:  # 2D
        xc = autocorr(x)  # default option: computes raw correlations with NO
        # normalization -- Matlab help on xcorr

        # Bias correction
        v1 = np.hstack((np.arange(1, n[0] + 1), np.arange(n[0] - 1, 0, -1)))[np.newaxis, :]
        v2 = np.hstack((np.arange(1, n[1] + 1), np.arange(n[1] - 1, 0, -1)))[np.newaxis, :]

        vd = np.dot(v1.T, v2)
        xc = xc / vd
        parzen_window_2D = np.dot(parzen_w_1, parzen_w_2.T)
        xc = xc * parzen_window_2D
        xf = fftshift(fft2(xc))

    elif x.ndim == 3 and min(n) != 1:  # 3D
        xc = np.zeros((2*n[0]-1, 2*n[1]-1, 2*n[2]-1))
        for m3 in range(n[2] - 1):
            temp = np.zeros((2*n[0]-1, 2*n[1]-1))
            for k in range(n[2] - m3):
                temp = temp + correlate2d(x[:, :, k + m3], x[:, :, k])
                # default option:
                # computes raw correlations with NO normalization
                # -- Matlab help on xcorr
            xc[:, :, (n[2] - 1) - m3] = temp
            xc[:, :, (n[2] - 1) + m3] = temp

        # Bias correction
        v1 = np.hstack((np.arange(1, n[0] + 1), np.arange(n[0] - 1, 0, -1)))[np.newaxis, :]
        v2 = np.hstack((np.arange(1, n[1] + 1), np.arange(n[1] - 1, 0, -1)))[np.newaxis, :]
        v3 = np.arange(n[2], 0, -1)

        vd = np.dot(v1.T, v2)
        vcu = np.zeros((2*n[0]-1, 2*n[1]-1, 2*n[2]-1))
        for m3 in range(n[2]):
            vcu[:, :, (n[2] - 1) - m3] = vd * v3[m3]
            vcu[:, :, (n[2] - 1) + m3] = vd * v3[m3]
        
        # Possible source of NAN values
        xc = xc / vcu

        parzen_window_2D = np.dot(parzen_w_1[np.newaxis, :].T, parzen_w_2[np.newaxis, :])
        parzen_window_3D = np.zeros((2*n[0]-1, 2*n[1]-1, 2*n[2]-1))
        for m3 in range(n[2] - 1):
            parzen_window_3D[:, :, (n[2] - 1) - m3] = np.dot(parzen_window_2D, parzen_w_3[n[2] - 1 - m3])
            parzen_window_3D[:, :, (n[2] - 1) + m3] = np.dot(parzen_window_2D, parzen_w_3[n[2] - 1 + m3])

        xc = xc * parzen_window_3D

        xf = fftshift(fftn(xc))

    else:

        print('ERROR!!!!! Unrecognized matrix dimension.')

    xf = abs(xf)
    xf[xf < 1e-4] = 1e-4
    out = 0.5 * np.log(2 * np.pi * np.exp(1)) + sumN(np.log(abs(
        (xf)))) / 2 / sumN(abs(xf))

    return out


def est_indp_sp(x):

    dimv = x.shape
    s0 = 0

    for j in range(np.min(dimv) - 1):
        x_sb = subsampling(x, j+1, [0, 0, 0])
        if j == 0:
            print('Estimating the entropy rate of the Gaussian component with subsampling depth {},'.format(j))
        else:
            print(' {},'.format(j))
        
        entrate_m = entrate_sp(x_sb, 1)

        ent_ref = 1.41
        if entrate_m > ent_ref:
            s0 = j
            break

    print(' Done;')
    if s0 == 0:
        print('ERROR!!!!! Ill conditioned data, can not estimate independent samples.(est_indp_sp)')
    else:
        s = s0

    return s, entrate_m

In [4]:
def subsampling(x,s,x0):
# Subsampling the data evenly with space 's'

    n = x.shape

    if x.ndim == 2 and np.min(n) == 1:  # 1D
        out = x[np.arange(x0[0], np.max(n), s)]
    
    elif x.ndim == 2 and np.min(n) != 1:  # 2D
        out = x[np.arange(x0[0], n[0], s),:][:,np.arange(x0[1], n[1], s)]
    
    elif x.ndim == 3 and np.min(n) != 1:  # 3D
        out = x[np.arange(x0[0], n[0], s),:,:][:,np.arange(x0[1], n[1], s),:][:,:,np.arange(x0[2], n[2], s)]
    
    else:
        print('ERROR!!!! Unrecognized matrix dimension!(subsampling)');
    
    return out

In [5]:
def kurtn(x):

    kurt = np.zeros((x.shape[1],1))

    for i in range(x.shape[1]):
        a = detrend(x[:, i], type='constant')
        a = a/np.std(a)
        kurt[i] = np.mean(a**4) - 3

    return kurt

In [6]:
def icatb_svd(data,numpc):
    _, Lambda, vh = svd(data)
    # Sort eigen vectors in Ascending order
    V = vh.T
    Lambda = Lambda / np.sqrt(data.shape[0] - 1) # Whitening (sklearn)
    inds = np.argsort(np.power(Lambda,2))
    Lambda = np.power(Lambda, 2)[inds]
    V = V[:, inds]
    sumAll = np.sum(Lambda)
    
    # Return only the extracted components
    V = V[:, (V.shape[1]-numpc):]
    Lambda = Lambda[Lambda.shape[0]-numpc:]
    sumUsed = np.sum(Lambda)
    retained = (sumUsed/sumAll)*100
    print('{ret}% of non-zero eigenvalues retained'.format(ret=retained))
    
    return V,Lambda

In [7]:
# PATH DATA
MASK_Path = '/Users/enekourunuela/tedana/docs/data/GIFT_mask.nii'
OC_Path   = '/Users/enekourunuela/tedana/docs/data/sbj.OCTS.nii'

In [8]:
# LOAD DATA
data_nib = nib.load(OC_Path).get_data()
mask_nib = nib.load(MASK_Path).get_data()
print(data_nib.shape)
print(mask_nib.shape)
[Nx,Ny,Nz,Nt] = data_nib.shape
data_nib_V = np.reshape(data_nib,(Nx*Ny*Nz,Nt), order='F')
maskvec = np.reshape(mask_nib,Nx*Ny*Nz, order='F')
DATA = data_nib_V[maskvec==1,:]
DATA.shape

(41, 52, 28, 160)
(41, 52, 28)


(24586, 160)

In [9]:
scaler = StandardScaler(with_mean=True, with_std=True)
data   = scaler.fit_transform(DATA) # This was X_sc

In [10]:
# SVD
numpc = Nt
_, Lambda, vh = svd(data)
# Sort eigen vectors in Ascending order
V = vh.T
Lambda = Lambda / np.sqrt(data.shape[0] - 1) # Whitening (sklearn)
inds = np.argsort(np.power(Lambda,2))
Lambda = np.power(Lambda, 2)[inds]
V = V[:, inds]
sumAll = np.sum(Lambda)

In [11]:
# Return only the extracted components
V = V[:, (V.shape[1]-numpc):]
Lambda = Lambda[Lambda.shape[0]-numpc:]
sumUsed = np.sum(Lambda)
retained = (sumUsed/sumAll)*100
print('{ret}% of non-zero eigenvalues retained'.format(ret=retained))

100.0% of non-zero eigenvalues retained


In [12]:
# Renaming to follow GIFT code in MATLAB
EigenValues = Lambda
EigenValues = EigenValues[::-1]
dataN = np.dot(data, V[:, ::-1])
# Potentially the small differences come from the different signs on V

In [13]:
# Using 12 gaussian components from middle, top and bottom gaussian components to determine the subsampling depth. Final subsampling depth is determined using median
kurtv1 = kurtn(dataN)
kurtv1[EigenValues > np.mean(EigenValues)] = 1000
idx_gauss = np.where(((kurtv1[:,0] < 0.3) * (EigenValues > np.finfo(float).eps)) == 1)[0] # DOUBT: make sure np.where is giving us just one tuple
idx = np.array(idx_gauss[:]).T
dfs = len(np.where(EigenValues > np.finfo(float).eps)[0]) # degrees of freedom
minTp = 12
if (len(idx) >= minTp):
    middle = int(np.round(len(idx)/2))
    idx = np.hstack([idx[0:4],idx[middle-1:middle+3],idx[-4:]])
elif not idx:
    minTp = np.min([minTp, dfs])
    idx = np.arange(dfs - minTp, dfs)
    
idx = np.unique(idx)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  # This is added back by InteractiveShellApp.init_path()


In [18]:
# Estimate the subsampling depth for effectively i.i.d. samples
mask_ND = np.reshape(maskvec, (Nx, Ny, Nz), order='F')
ms = len(idx)
s = np.zeros((ms,))
for i in range(ms):
    x_single = np.zeros(Nx*Ny*Nz)
    x_single[maskvec==1] = dataN[:,idx[i]]
    x_single = np.reshape(x_single,(Nx,Ny,Nz),order='F')
    s[i] = est_indp_sp(x_single)[0]
    if i > 6:
        tmpS       = s[0:i]
        tmpSMedian = np.round(np.median(tmpS))
        if np.sum(tmpS == tmpSMedian) > 6:
            s = tmpS
            break
    dim_n = x_single.ndim
s1 = int(np.round(np.median(s)))
if np.floor(np.power(np.sum(maskvec)/Nt,1/dim_n)) < s1:
    s1 = int(np.floor(np.power(np.sum(maskvec)/Nt,1/dim_n)))
N = np.round(np.sum(maskvec)/np.power(s1,dim_n))

Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;
Estimating the entropy rate of the Gaussian component with subsampling depth 0,
 1,
 Done;


In [19]:
# Use the subsampled dataset to calculate eigen values
print('Perform EVD on the effectively i.i.d. samples ...')

if s1 != 1:
    mask_s    = subsampling(mask_ND,s1,[0,0,0])
    mask_s_1d = np.reshape(mask_s,np.prod(mask_s.shape),order='F')
    dat       = np.zeros((int(np.sum(mask_s_1d)),Nt))
    for i in range(Nt):
        x_single                = np.zeros((Nx*Ny*Nz,))
        x_single[maskvec==1]    = data[:,i]
        x_single                = np.reshape(x_single,(Nx,Ny,Nz),order='F')
        dat0                    = subsampling(x_single,s1,[0,0,0])
        dat0                    = np.reshape(dat0,np.prod(dat0.shape),order='F')
        dat[:,i]                = dat0[mask_s_1d==1]
        
    # Perform Variance Normalization
    dat  = scaler.fit_transform(dat)
    
    # (completed)
    print('P2:')
    [V, EigenValues] = icatb_svd(dat, Nt)
    EigenValues = EigenValues[::-1]
    
lam = EigenValues
print(' Effective number of i.i.d. samples %d' % N)

Perform EVD on the effectively i.i.d. samples ...
 Effective number of i.i.d. samples 24586


In [21]:
#eigensp_adj (309)
n = N
p = EigenValues.shape[0]
r = p/n
bp = np.power((1+np.sqrt(r)),2)
bm = np.power((1-np.sqrt(r)),2)
vv_step = (bp-bm)/(5*p-1)
vv = np.arange(bm,bp+vv_step,vv_step)
gv = (1/(2*np.pi*r*vv)) * np.sqrt(abs((vv-bm)*(bp-vv)))
gvd = np.zeros(gv.shape)
for i in range(gv.shape[0]):
    gvd[i] = sum(gv[0:i])
    
gvd = gvd/np.max(gvd)

lam_emp = np.zeros(lam.shape)
for i in range(p):
    i_norm = i/p
    minx = np.argmin(abs(i_norm-gvd))
    lam_emp[i] = vv[minx]

lam_emp = rot90(lam_emp, 2)

lam_adj = lam/lam_emp

0.8451662842274329


In [61]:
gv = (1/(2*np.pi*r*vv)) * np.sqrt(abs((vv-bm)*(bp-vv)))
gvd = np.zeros(gv.shape)

In [62]:
gv.shape

(800,)

In [ ]:
gv = 1./(2*pi*r*vv).*sqrt((vv-bm).*(bp-vv))

In [51]:
np.sqrt(-2.47251400e-14)

/opt/anaconda3/envs/tedana-hack/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in sqrt
  """Entry point for launching an IPython kernel.


nan

In [ ]:
def eigensp_adj(lam,n,p):
    r = p/n
    
    bp = np.power((1+np.sqrt(r)),2)
    bm = np.power((1-np.sqrt(r)),2)
    
    
    
    return lam_adj

In [ ]:
function lam_adj = eigensp_adj(lam,n,p)
% Eigen spectrum adjustment for EVD on finite samples

r = p/n;

bp = (1+sqrt(r))^2;
bm = (1-sqrt(r))^2;

vv = [bm:(bp-bm)/(5*p-1):bp];

gv = 1./(2*pi*r*vv).*sqrt((vv-bm).*(bp-vv));

gvd = zeros(size(gv));
for i = 1:length(gv)
    gvd(i) = sum(gv(1:i));
end
gvd = gvd/max(gvd);

lam_emp = zeros(size(lam));
for i = 1:p
    i_norm = i/p;
    [minv,minx] = min(abs(i_norm-gvd));
    lam_emp(i) = vv(minx);
end

lam_emp = rot90(lam_emp, 2);

lam_adj = lam./lam_emp;

In [17]:
# Make eigen spectrum adjustment
print(' Perform eigen spectrum adjustment ...')
lam = eigensp_adj(lam, N, length(lam));

In [18]:
print('P2:')
[V, EigenValues] = icatb_svd(dat, Nt)
EigenValues = EigenValues[::-1]

100.0% of non-zero eigenvalues retained


array([1.59979250e+02, 4.20888454e-02, 6.46422900e-03, 4.67639325e-03,
       2.61807413e-03, 2.51092920e-03, 1.70194562e-03, 1.34915374e-03,
       9.31375489e-04, 8.32087180e-04, 7.64005948e-04, 6.59946109e-04,
       5.18911647e-04, 4.71221549e-04, 4.19183571e-04, 4.02188504e-04,
       3.10550991e-04, 3.00879491e-04, 2.83793360e-04, 2.59717967e-04,
       2.31474156e-04, 2.22881701e-04, 2.09583134e-04, 1.61945234e-04,
       1.57916638e-04, 1.53905018e-04, 1.34948313e-04, 1.26360672e-04,
       1.22180363e-04, 1.12150334e-04, 1.07587022e-04, 1.05477193e-04,
       9.79176761e-05, 9.32848961e-05, 8.57886449e-05, 8.54593180e-05,
       8.25672011e-05, 7.79111767e-05, 7.39613576e-05, 7.08517622e-05,
       6.98959200e-05, 6.86650549e-05, 6.63949599e-05, 6.20013219e-05,
       6.08225548e-05, 5.97321849e-05, 5.84065686e-05, 5.57416123e-05,
       5.36666895e-05, 5.28873411e-05, 5.09026890e-05, 4.94383985e-05,
       4.63258299e-05, 4.48996790e-05, 4.38278844e-05, 4.27373519e-05,
      

In [21]:
EigenValues

array([4.72371663e-06, 5.01608286e-06, 5.11055498e-06, 5.17791046e-06,
       5.40969073e-06, 5.56323037e-06, 5.64816987e-06, 5.73338938e-06,
       6.02165874e-06, 6.05148177e-06, 6.17740071e-06, 6.43633540e-06,
       6.47980850e-06, 6.65694107e-06, 6.81826308e-06, 6.85028526e-06,
       7.02184581e-06, 7.17654477e-06, 7.33138865e-06, 7.62523656e-06,
       7.86083253e-06, 8.07312137e-06, 8.35380142e-06, 8.54934441e-06,
       8.66562337e-06, 8.72388732e-06, 9.13167526e-06, 9.37045929e-06,
       9.54790234e-06, 9.78507220e-06, 1.02345978e-05, 1.04939838e-05,
       1.08155665e-05, 1.10924479e-05, 1.15284610e-05, 1.18855342e-05,
       1.19528651e-05, 1.21668868e-05, 1.24231229e-05, 1.27773521e-05,
       1.30784724e-05, 1.33147241e-05, 1.37861544e-05, 1.41180821e-05,
       1.45025987e-05, 1.48035400e-05, 1.53477275e-05, 1.55966088e-05,
       1.59137805e-05, 1.60126403e-05, 1.65266750e-05, 1.69744350e-05,
       1.72073990e-05, 1.73609840e-05, 1.77185460e-05, 1.80799787e-05,
      

In [ ]:
# (completed)
print('P2:')
[V, EigenValues] = icatb_svd(dat, Nt);
EigenValues = diag(EigenValues);
EigenValues = EigenValues(end:-1:1);

***

In [13]:
i=0
x_single = np.zeros(Nx*Ny*Nz)
x_single[maskvec==1] = dataN[:,idx[i]]
x_single = np.reshape(x_single,(Nx,Ny,Nz),order='F')



In [14]:
from scipy.io import loadmat
MATLAB_Results = loadmat('./x_single_mat.mat')
out_mat = loadmat('./out_mat.mat')['out']
MATLAB_X_single = MATLAB_Results['x']

In [15]:
np.corrcoef(np.reshape(x_single,np.prod(x_single.shape),order='F'),np.reshape(MATLAB_X_single,np.prod(MATLAB_X_single.shape),order='F'))

array([[1.        , 0.99999881],
       [0.99999881, 1.        ]])

In [19]:
import xarray as xr
x0 = [0,0,0]
x_1d = np.squeeze(x_single[0,0,:])
n = x_single.shape
s = 0
out = x_single[np.arange(x0[0], n[0], s+1),:,:][:,np.arange(x0[1], n[1], s+1),:][:,:,np.arange(x0[2], n[2], s+1)]

In [ ]:
def est_indp_sp(x):
    dimv = x.shape
    s0 = 0
    for j in range(np.min(dimv)-1):
        x_sb = subsampling(x,j,[0,0,0]);
        if j == 1:
            fprintf('\n Estimating the entropy rate of the Gaussian component with subsampling depth %d,',j);
        else
            fprintf(' %d,',j);
        entrate_m = entrate_sp(x_sb,1);
    
        ent_ref = 1.41;
        if entrate_m > ent_ref:
            s0 = j; break;
    fprintf(' Done;');
    
    if s0 == 0:
        error('Ill conditioned data, can not estimate independent samples.(est_indp_sp)');
    else:
        s = s0;
    
    return s, entrate_m

In [65]:
for i in range(ms):
    x_single = np.zeros(Nx*Ny*Nz)
    x_single[maskvec==1] = dataN[:,idx[i]]
    x_single = np.reshape(x_single,(Nx,Ny,Nz),order='F')
    s(i) = est_indp_sp(x_single)
    if (i > 6):
        tmpS = s(1:i);
        tmpSMedian = round(median(tmpS));
        if (length(find(tmpS == tmpSMedian)) > 6):
            s = tmpS;
            break
    dim_n = np.prod(size(size(x_single)))
    del x_single

12

In [116]:
x0 = [0,0,0]
x_1d = np.squeeze(x_single[0,0,:])
n = x_single.shape
s = 0
# x[np.arange(x0[0],np.max(n), s+1)] 1D
out = x_single[np.arange(x0[0], n[0], s+1),:,:][:,np.arange(x0[1], n[1], s+1),:][:,:,np.arange(x0[2], n[2], s+1)]

In [150]:
x_single[[np.arange(x0[0], n[0], s+1)],[np.arange(x0[1], n[1], s+1)],[np.arange(x0[2], n[2], s+1)]]

IndexError: shape mismatch: indexing arrays could not be broadcast together with shapes (1,41) (1,52) (1,28) 

In [166]:
import xarray as xr
JAVI = xr.DataArray(x_single,dims=['x','y','z'],coords={'x':np.arange(Nx),'y':np.arange(Ny),'z':np.arange(Nz)}, name='x_single')

In [176]:
JAVI_SEL = JAVI.isel(x=np.arange(x0[0], n[0], s+1), y=np.arange(x0[1], n[1], s+1),z=np.arange(x0[2], n[2], s+1))

In [178]:
np.corrcoef(np.reshape(JAVI_SEL.values,np.prod(JAVI_SEL.values.shape),order='F'),np.reshape(out_mat,np.prod(out_mat.shape),order='F'))

array([[ 1.        , -0.00532732],
       [-0.00532732,  1.        ]])

In [182]:
np.corrcoef(np.reshape(x_single,np.prod(x_single.shape),order='F'),np.reshape(MATLAB_X_single,np.prod(MATLAB_X_single.shape),order='F'))

array([[ 1.        , -0.00532732],
       [-0.00532732,  1.        ]])

In [123]:
np.corrcoef(np.reshape(out,np.prod(out.shape),order='F'),np.reshape(out_mat,np.prod(out_mat.shape),order='F'))

array([[ 1.        , -0.00532732],
       [-0.00532732,  1.        ]])

In [128]:
np.arange(x0[2], n[2], s+1)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27])

In [113]:
(x_single[np.array([1,2,3]),:,:][:,np.array([5,6,7,8,9,10]),:][:,:,np.array([20,21])]).shape

(3, 6, 2)

In [125]:
np.max(out)

0.02060704305768013

In [179]:
from scipy.io import loadmat
MATLAB_Results = loadmat('./x_single_mat.mat')
out_mat = loadmat('./out_mat.mat')['out']

In [181]:
MATLAB_X_single = MATLAB_Results['x']

In [40]:
MATLAB_dataN = np.loadtxt('./dataN_matlab.txt')

In [43]:
np.corrcoef(np.reshape(dataN,np.prod(dataN.shape),order='F'),np.reshape(MATLAB_dataN,np.prod(MATLAB_dataN.shape),order='F'))

array([[1.        , 0.99994707],
       [0.99994707, 1.        ]])

In [52]:
# THIS WORKS
i=0
J_x_single = np.zeros(Nx*Ny*Nz)
J_x_single[maskvec==1] = dataN[:,idx[i]]
J_x_single = np.reshape(J_x_single,(Nx,Ny,Nz),order='F')

In [53]:
np.corrcoef(np.reshape(MATLAB_X_single,np.prod(MATLAB_X_single.shape),order='F'),np.reshape(J_x_single,np.prod(J_x_single.shape),order='F'))

array([[1.        , 0.99999881],
       [0.99999881, 1.        ]])

In [77]:
x_single.shape

(41, 52, 28)

array([[ 1.        , -0.00532732],
       [-0.00532732,  1.        ]])

In [49]:
np.hstack([j[0:4],j[middle-1:middle+3],j[-4:]])

array([  0,   1,   2,   3,  79,  80,  81,  82, 156, 157, 158, 159])

In [55]:
j = np.arange(dfs - minTp, dfs)
j

array([148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159])

In [25]:
n = x_single.shape
[int(i) for i in np.ceil(np.array(n) / 10)]

[5, 6, 3]

(41, 52, 28)